# FP16 GGUF → Q5_K_M 量子化ガイド

## 概要

[grapevine-AI/Qwen2.5-Coder-32B-Instruct-GGUF](https://huggingface.co/grapevine-AI/Qwen2.5-Coder-32B-Instruct-GGUF) の FP16 モデルから Q5_K_M を作成する手順です。

### 前提条件

| 項目 | 要件 |
|------|------|
| **Colab プラン** | Pay As You Go |
| **ランタイム** | A100 GPU（ディスク容量・RAM確保のため） |
| **Google One** | 2TB 契約中（Google Drive に出力を保存） |
| **HuggingFace** | アカウント（トークン推奨、なくてもダウンロード可能な場合あり） |

### ファイルサイズ見積もり

| ファイル | サイズ |
|---------|--------|
| FP16 GGUF（入力） | 約 65 GB（分割ファイル） |
| Q5_K_M GGUF（出力） | 約 23 GB |
| llama.cpp ビルド | 約 1 GB |
| **合計必要ディスク** | **約 90 GB** |

A100 ランタイムのローカルディスクは約 200GB あるため十分です。

---
## Step 0: ランタイム設定

**実行前に必ず以下を確認してください：**

1. メニュー → `ランタイム` → `ランタイムのタイプを変更`
2. **ハードウェアアクセラレータ**: `A100 GPU` を選択
3. 接続して、このノートブックを上から順に実行

> **なぜA100か？**  
> GPUを使うためではなく、A100ランタイムはディスク容量（約200GB）とシステムRAM（約83GB）が大きいため。  
> 32Bモデルの FP16 は約65GB あるので、標準ランタイム（約78GBディスク）では容量不足になります。  
> L4 ランタイムでも動作する可能性がありますが、ディスク容量が足りるか確認が必要です。

---
## Step 1: 環境確認

In [ ]:
# ディスク容量とRAMを確認
!df -h /
print("---")
!free -h
print("---")
# GPU確認（量子化自体にGPUは不要だが、ランタイム種別の確認用）
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || echo "GPU なし（CPUランタイム）"

---
## Step 2: Google Drive マウント

完成した Q5_K_M ファイル（約23GB）を Google Drive に保存するためにマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 出力先ディレクトリを作成
!mkdir -p /content/drive/MyDrive/gguf-models
print("Google Drive マウント完了。出力先: /content/drive/MyDrive/gguf-models/")

---
## Step 3: llama.cpp のビルド

`llama-quantize` コマンドを使うために llama.cpp をソースからビルドします。

In [ ]:
# ビルド用の依存パッケージをインストール
!apt-get update -qq && apt-get install -y -qq build-essential cmake git > /dev/null 2>&1
print("依存パッケージインストール完了")

In [ ]:
# llama.cpp をクローンしてビルド
%cd /content
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git
%cd /content/llama.cpp

# CMake でビルド（CPU のみで十分。量子化に GPU は不要）
!cmake -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build --config Release -j$(nproc)

print("\n=== ビルド完了 ===")
# llama-quantize の存在を確認
!ls -lh /content/llama.cpp/build/bin/llama-quantize
# llama-gguf-split の存在を確認（分割ファイルのマージ用）
!ls -lh /content/llama.cpp/build/bin/llama-gguf-split

---
## Step 4: FP16 モデルのダウンロード

grapevine-AI リポジトリから FP16 GGUF ファイルをダウンロードします。

32Bモデルの FP16 は約 65GB あり、HuggingFace の制限により**複数ファイルに分割**されている可能性があります。

In [ ]:
# huggingface_hub をインストール
!pip install -q huggingface_hub[hf_transfer]

# hf_transfer で高速ダウンロードを有効化
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
# （任意）HuggingFace にログイン
# プライベートリポジトリの場合やレート制限回避のために必要
# トークンは https://huggingface.co/settings/tokens で作成できます

# from huggingface_hub import login
# login(token="hf_xxxxx")  # ← 自分のトークンに置き換え

In [ ]:
# FP16 ファイルをダウンロード
# --include で fp16 関連ファイルのみを指定
!huggingface-cli download \
    grapevine-AI/Qwen2.5-Coder-32B-Instruct-GGUF \
    --include "*fp16*" \
    --local-dir /content/fp16-model \
    --local-dir-use-symlinks False

print("\n=== ダウンロード完了 ===")
!ls -lh /content/fp16-model/*.gguf

In [ ]:
# ダウンロードされたファイルの確認
import glob

fp16_files = sorted(glob.glob("/content/fp16-model/*fp16*.gguf"))
print(f"FP16 ファイル数: {len(fp16_files)}")
for f in fp16_files:
    size_gb = os.path.getsize(f) / (1024**3)
    print(f"  {os.path.basename(f)} ({size_gb:.2f} GB)")

total_gb = sum(os.path.getsize(f) for f in fp16_files) / (1024**3)
print(f"\n合計サイズ: {total_gb:.2f} GB")

# 分割ファイルか単一ファイルかを判定
is_split = len(fp16_files) > 1
print(f"分割ファイル: {'はい' if is_split else 'いいえ'}")

---
## Step 5: 分割ファイルのマージ（必要な場合のみ）

FP16 が複数ファイルに分割されている場合、`llama-gguf-split --merge` で結合します。  
単一ファイルの場合はこのセルはスキップされます。

> **注意**: マージ中は FP16 の分割ファイル + マージ後のファイルで一時的にディスク使用量が倍増します（約130GB）。  
> A100 ランタイム（約200GBディスク）なら問題ありません。

In [ ]:
import subprocess

QUANTIZE_BIN = "/content/llama.cpp/build/bin/llama-quantize"
GGUF_SPLIT_BIN = "/content/llama.cpp/build/bin/llama-gguf-split"

fp16_files = sorted(glob.glob("/content/fp16-model/*fp16*.gguf"))
is_split = len(fp16_files) > 1

if is_split:
    # 分割ファイルの場合：最初のシャードを指定してマージ
    first_shard = fp16_files[0]
    merged_path = "/content/fp16-model/merged-fp16.gguf"
    print(f"分割ファイルをマージします...")
    print(f"  入力: {first_shard}")
    print(f"  出力: {merged_path}")

    result = subprocess.run(
        [GGUF_SPLIT_BIN, "--merge", first_shard, merged_path],
        capture_output=True, text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print(f"エラー: {result.stderr}")
        # 最近の llama-quantize は分割ファイルを直接扱えるため、
        # マージ失敗時は分割ファイルの最初のシャードを直接使用
        print("\n→ マージに失敗しました。分割ファイルを直接 quantize に渡します。")
        INPUT_MODEL = first_shard
    else:
        size_gb = os.path.getsize(merged_path) / (1024**3)
        print(f"\nマージ完了: {size_gb:.2f} GB")
        INPUT_MODEL = merged_path

        # マージ後、分割ファイルを削除してディスク容量を確保
        print("分割ファイルを削除してディスク容量を確保...")
        for f in fp16_files:
            os.remove(f)
        print("削除完了")
else:
    INPUT_MODEL = fp16_files[0]
    print(f"単一ファイルです: {INPUT_MODEL}")

print(f"\n量子化に使用するファイル: {INPUT_MODEL}")
!df -h /

---
## Step 6: Q5_K_M に量子化

llama.cpp の `llama-quantize` を使って FP16 → Q5_K_M に変換します。

### 量子化タイプの解説

| タイプ | サイズ (32B) | 品質 | 説明 |
|--------|-------------|------|------|
| Q4_K_M | ~19 GB | 良い | バランス型。VRAM が限られる場合に最適 |
| **Q5_K_M** | **~23 GB** | **とても良い** | **FP16 に近い品質。推奨** |
| Q6_K | ~27 GB | 優秀 | ほぼ FP16 品質だがサイズ大 |
| Q8_0 | ~34 GB | 最高 | 量子化の中では最高品質 |

> `K_M` は K-quant の Medium バリアント。重要なレイヤーにはより高い精度を割り当てます。

In [ ]:
import time

OUTPUT_MODEL = "/content/Qwen2.5-Coder-32B-Instruct-Q5_K_M.gguf"
QUANT_TYPE = "Q5_K_M"

print(f"=== 量子化開始 ===")
print(f"  入力: {INPUT_MODEL}")
print(f"  出力: {OUTPUT_MODEL}")
print(f"  量子化タイプ: {QUANT_TYPE}")
print(f"  ※ 32Bモデルの場合、20〜40分程度かかります")
print()

start = time.time()

!{QUANTIZE_BIN} \
    {INPUT_MODEL} \
    {OUTPUT_MODEL} \
    {QUANT_TYPE}

elapsed = time.time() - start
print(f"\n=== 量子化完了 ===")
print(f"所要時間: {elapsed/60:.1f} 分")

if os.path.exists(OUTPUT_MODEL):
    size_gb = os.path.getsize(OUTPUT_MODEL) / (1024**3)
    print(f"出力ファイルサイズ: {size_gb:.2f} GB")
else:
    print("エラー: 出力ファイルが生成されませんでした")

---
## Step 7: Google Drive に保存

生成した Q5_K_M ファイルを Google Drive にコピーします。  
Colab のランタイムは時間制限があるため、成果物を必ず保存してください。

In [ ]:
import shutil

DRIVE_OUTPUT = "/content/drive/MyDrive/gguf-models/Qwen2.5-Coder-32B-Instruct-Q5_K_M.gguf"

print(f"Google Drive にコピー中...")
print(f"  元: {OUTPUT_MODEL}")
print(f"  先: {DRIVE_OUTPUT}")
print(f"  ※ 約23GBのコピーには数分かかります")

start = time.time()
shutil.copy2(OUTPUT_MODEL, DRIVE_OUTPUT)
elapsed = time.time() - start

drive_size = os.path.getsize(DRIVE_OUTPUT) / (1024**3)
print(f"\nコピー完了！ ({elapsed/60:.1f} 分)")
print(f"保存先: {DRIVE_OUTPUT} ({drive_size:.2f} GB)")

---
## Step 8: クリーンアップ（任意）

ローカルディスクの一時ファイルを削除します。

In [ ]:
# FP16 モデルの削除（ローカルディスクの容量確保）
!rm -rf /content/fp16-model
!rm -f /content/Qwen2.5-Coder-32B-Instruct-Q5_K_M.gguf
print("ローカルの一時ファイルを削除しました")
print("Q5_K_M ファイルは Google Drive に保存済みです")
!df -h /

---
## 補足: HuggingFace にアップロードする場合（任意）

作成した Q5_K_M を自分の HuggingFace リポジトリにアップロードできます。

In [ ]:
# # HuggingFace にアップロード（必要な場合のみコメント解除）
# from huggingface_hub import HfApi, login
#
# # ログイン（Write権限のあるトークンが必要）
# login(token="hf_xxxxx")
#
# api = HfApi()
#
# # リポジトリ作成（存在しない場合）
# repo_id = "your-username/Qwen2.5-Coder-32B-Instruct-Q5_K_M-GGUF"
# api.create_repo(repo_id, exist_ok=True, repo_type="model")
#
# # アップロード
# api.upload_file(
#     path_or_fileobj=DRIVE_OUTPUT,
#     path_in_repo="Qwen2.5-Coder-32B-Instruct-Q5_K_M.gguf",
#     repo_id=repo_id,
# )
# print(f"アップロード完了: https://huggingface.co/{repo_id}")

---
## トラブルシューティング

### ディスク容量不足

```
Error: failed to open file
```

→ `df -h /` でディスク残量を確認。FP16ダウンロード前にローカルディスクに70GB以上の空きが必要です。  
→ A100ランタイムに切り替えるか、不要なファイルを削除してください。

### メモリ不足（OOM）

```
std::bad_alloc / Killed
```

→ llama-quantize はレイヤー単位で処理するため通常は問題ありませんが、発生した場合は「ハイメモリ」ランタイムを選択してください。

### ダウンロードが途中で止まる

→ `huggingface-cli download` はレジューム対応です。同じコマンドを再実行すれば途中から再開されます。

### llama-quantize が見つからない

→ llama.cpp のビルドに失敗している可能性があります。Step 3 を再実行してください。  
→ パスが変わっている場合: `!find /content/llama.cpp/build -name 'llama-quantize'` で場所を確認。

### 分割ファイルの直接量子化

最近の llama.cpp では、分割 GGUF を直接 `llama-quantize` に渡せます（最初のシャードを指定）。  
マージが失敗する場合はこの方法を試してください。